In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import scipy as sp
import anndata
import gc
import matplotlib as mpl
import scipy
from matplotlib import pyplot as plt
import loompy
import warnings
warnings.filterwarnings('ignore')

ds = loompy.connect("D:\\Data\\stereo-seq\\l5_all.loom")
st_adata=anndata.read_h5ad('D:\\Data\\Slide-seqV2\\mouse_hippocampus\\Puck_200115_08.h5ad')
marker_df = pd.read_excel('D:/Data/stereo-seq/mmc4.xlsx', sheet_name=None)

#ds = loompy.connect("l5_all.loom")
#st_adata=anndata.read_h5ad('Puck_200115_08.h5ad')
#marker_df = pd.read_excel('mmc4.xlsx', sheet_name=None)

In [2]:
sc.pp.highly_variable_genes(st_adata, flavor='seurat_v3',n_top_genes=5000)

In [3]:
high_gene_lst = []
for i in range(len(st_adata.var['highly_variable'])):
    flag = list(st_adata.var['highly_variable'])[i]
    if flag is True:
        high_gene_lst.append(list(st_adata.var_names)[i])

In [4]:
st_gene_list = list(st_adata.var_names)
marker_gene_lst = []
gene_marker_dict = {} 
for index, row in marker_df['markers_spec_selec_rob_10-May-2'].iterrows():
    if row['Unnamed: 1'] == 'Marker':
        ct = row['Cluster ID']
        marker_lst = [row['Gene1'], row['Gene2'], row['Gene3'], row['Gene4'], row['Gene5']]
        gene_marker_dict.update({ct : marker_lst})
        for gene in marker_lst:
            if gene not in marker_gene_lst and gene in st_gene_list:
                marker_gene_lst.append(gene)

In [5]:
len(marker_gene_lst)

962

In [6]:
from collections import Counter

count_ct_dict = Counter(ds.ca['ClusterName'])

filter_ct = []
for ct in count_ct_dict:
    if count_ct_dict[ct] > 50 and ct in ds.ca['ClusterName']:
        filter_ct.append(ct)

In [7]:
tmp_filter_cell_idx_lst = []
tmp_filter_cell_lst = []
for i in range(len(ds.ca['ClusterName'])):
    if ds.ca['ClusterName'][i] in filter_ct:
        tmp_filter_cell_idx_lst.append(i)
        tmp_filter_cell_lst.append(ds.ca['CellID'][i])
        


In [8]:
tmp_common_gene_lst = []
tmp_ls = []
for gene in st_gene_list:
    if gene in ds.ra.Gene.tolist():
        if gene in high_gene_lst or gene in marker_gene_lst:
            gene_idx = ds.ra.Gene.tolist().index(gene)
            tmp_ls.append(ds[gene_idx, tmp_filter_cell_idx_lst])
            tmp_common_gene_lst.append(gene)

In [9]:
from scipy import sparse
expr_matrix = sparse.csr_matrix(np.array(tmp_ls).T)
adata = anndata.AnnData(expr_matrix)
adata.obs_names = tmp_filter_cell_lst
adata.var_names = tmp_common_gene_lst

gc.collect()

17137

In [10]:
adata.obs['celltype'] = ds.ca['ClusterName'][tmp_filter_cell_idx_lst]

tmp_type_top_genes_dict ={}
sc.tl.rank_genes_groups(adata, 'celltype', method='wilcoxon')
for cell_type in filter_ct:
    tmp_type_top_genes_dict.update({cell_type:pd.DataFrame(adata.uns['rank_genes_groups']['names'])[cell_type].tolist()})

... storing 'celltype' as categorical


In [11]:
scanpy_marker_ls = []
for ct in tmp_type_top_genes_dict:
    for gene in tmp_type_top_genes_dict[ct][:50]:
        if type(gene) != type('str'):
            continue
        if gene not in scanpy_marker_ls:
            scanpy_marker_ls.append(gene)

In [12]:
sc_gene_lst = ds.ra.Gene.tolist()

In [13]:
marker_genes = []
for gene in st_gene_list:
    if gene in sc_gene_lst:
        if gene in marker_gene_lst or gene in scanpy_marker_ls:
            marker_genes.append(gene)
len(marker_genes)

1841